In [1]:
from senticnet.senticnet import SenticNet
sn = SenticNet()
text = 'corona'
concept_info = sn.concept(text)
print(concept_info)

{'polarity_value': 'positive', 'polarity_intense': '0.822', 'moodtags': ['#joy', '#surprise'], 'sentics': {'pleasantness': '0.931', 'attention': '-0.82', 'sensitivity': '0', 'aptitude': '0.709'}, 'semantics': ['light', 'aureole', 'radiance', 'visible_light', 'visible_radiation']}


In [3]:
from senticnet.senticnet import SenticNet
sn = SenticNet()
text = 'corona_virus'
concept_info = sn.concept(text)
print(concept_info)

KeyError: 'corona_virus'

In [1]:
from senticnet.senticnet import SenticNet
sn = SenticNet()
text = 'loving'
concept_info = sn.concept(text)
print(concept_info)

{'polarity_value': 'positive', 'polarity_intense': '0.287', 'moodtags': ['#joy', '#interest'], 'sentics': {'pleasantness': '0.494', 'attention': '0.673', 'sensitivity': '-0.26', 'aptitude': '0.25'}, 'semantics': ['adorably', 'lovingly', 'fondly', 'endearingly', 'devotedness']}


In [2]:
from senticnet.senticnet import SenticNet
sn = SenticNet()
text = 'love'
concept_info = sn.concept(text)
print(concept_info)

{'polarity_value': 'positive', 'polarity_intense': '0.83', 'moodtags': ['#joy', '#admiration'], 'sentics': {'pleasantness': '1', 'attention': '0', 'sensitivity': '0', 'aptitude': '0.66'}, 'semantics': ['tenderness', 'affection', 'kindness', 'care', 'passion']}


In [3]:
from senticnet.senticnet import SenticNet
sn = SenticNet()
text = 'coding'
concept_info = sn.concept(text)
print(concept_info)

{'polarity_value': 'positive', 'polarity_intense': '0.782', 'moodtags': ['#surprise', '#admiration'], 'sentics': {'pleasantness': '0.634', 'attention': '-0.73', 'sensitivity': '0', 'aptitude': '0.974'}, 'semantics': ['decompression', 'cryptography', 'decipherment', 'secret_writing', 'steganography']}


In [6]:
from senticnet.senticnet import SenticNet
sn = SenticNet()
text = 'self love'
concept_info = sn.concept(text)
print(concept_info)

{'polarity_value': 'negative', 'polarity_intense': '-0.82', 'moodtags': ['#anger', '#disgust'], 'sentics': {'pleasantness': '0', 'attention': '0', 'sensitivity': '0.747', 'aptitude': '-0.90'}, 'semantics': ['conceit', 'amour_propre', 'self_pride', 'egotism', 'self-importance']}


In [7]:
from senticnet.senticnet import SenticNet
sn = SenticNet()
text = 'self'
concept_info = sn.concept(text)
print(concept_info)

{'polarity_value': 'positive', 'polarity_intense': '0.854', 'moodtags': ['#joy', '#interest'], 'sentics': {'pleasantness': '0.825', 'attention': '0.883', 'sensitivity': '0', 'aptitude': '0'}, 'semantics': ['ego', 'research', 'messageid', 'symposium', 'behavior']}


In [8]:
from senticnet.senticnet import SenticNet
sn = SenticNet()
text = 'love'
concept_info = sn.concept(text)
print(concept_info)

{'polarity_value': 'positive', 'polarity_intense': '0.83', 'moodtags': ['#joy', '#admiration'], 'sentics': {'pleasantness': '1', 'attention': '0', 'sensitivity': '0', 'aptitude': '0.66'}, 'semantics': ['tenderness', 'affection', 'kindness', 'care', 'passion']}
